In [1]:
#use data_team_concurrent iam
dbutils.fs.cp("s3://ehth-databricks-poc-test/ehealth/JingP/cora/cora.cites","dbfs:/tmp/cora/cora.cites")
dbutils.fs.cp("s3://ehth-databricks-poc-test/ehealth/JingP/cora/cora.content","dbfs:/tmp/cora/cora.content")

Out[1]: True

In [2]:
#then use horovod cluster
dbutils.fs.cp("dbfs:/tmp/cora/cora.cites","file:/tmp/cora/cora.cites")
dbutils.fs.cp("dbfs:/tmp/cora/cora.content","file:/tmp/cora/cora.content")

Out[1]: True

In [3]:
%fs ls /tmp/cora/

path,name,size
dbfs:/tmp/cora/cora.cites,cora.cites,69928
dbfs:/tmp/cora/cora.content,cora.content,7823427


In [4]:
%sh ls /tmp/cora/

cora.cites
cora.content

In [5]:
%run ./utils

In [6]:
%run ./layers/graph

In [7]:
from __future__ import print_function

from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

#from kegra.layers.graph import GraphConvolution
#from kegra.utils import *

import time

# Define parameters
DATASET = 'cora'
PATH='/tmp/cora/'
FILTER = 'localpool'  # 'chebyshev'
MAX_DEGREE = 2  # maximum polynomial degree
SYM_NORM = True  # symmetric (True) vs. left-only (False) normalization
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience


# Get data
X, A, y = load_data(path=PATH,dataset=DATASET)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

# Normalize X
X /= X.sum(1).reshape(-1, 1)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.

In [8]:
if FILTER == 'localpool':
    """ Local pooling filters (see 'renormalization trick' in Kipf & Welling, arXiv 2016) """
    print('Using local pooling filters...')
    A_ = preprocess_adj(A, SYM_NORM)
    support = 1
    graph = [X, A_]
    G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

elif FILTER == 'chebyshev':
    """ Chebyshev polynomial basis filters (Defferard et al., NIPS 2016)  """
    print('Using Chebyshev polynomial basis filters...')
    L = normalized_laplacian(A, SYM_NORM)
    L_scaled = rescale_laplacian(L)
    T_k = chebyshev_polynomial(L_scaled, MAX_DEGREE)
    support = MAX_DEGREE + 1
    graph = [X]+T_k
    G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True) for _ in range(support)]

else:
    raise Exception('Invalid filter type.')

X_in = Input(shape=(X.shape[1],))

Using local pooling filters...

In [9]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(16, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

# Compile model
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

# Testing
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print("Test set results:",
      "loss= {:.4f}".format(test_loss[0]),
      "accuracy= {:.4f}".format(test_acc[0]))

WARNING:tensorflow:From /databricks/python/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:423: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
WARNING:tensorflow:From /databricks/python/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
WARNING:tensorflow:From /databricks/python/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
Epoch: 0001 train_loss= 1.9356 train_acc= 0.3000 val_loss= 1.9372 val_acc= 0.2733 time= 1.1133
Epoch: 0002 train_loss= 1.9230 train_acc= 0.3500 val_loss= 1.9264 val_acc= 0.2767 time= 0.0811
Epoch: 0003 train_loss= 1.9092 train_acc= 0.3643 val_loss= 1.9148 val_acc= 0.2800 time= 0.0802
Epoch: 0004 train_loss= 1.8943 train_acc= 0.3857 val_loss= 1.9024 val_acc= 0.2833 time= 0.0784
Epoch: 0005 train_loss= 1.8785 train_acc= 0.3929 val_loss= 1.8895 val_acc= 0.2800 time= 0.0789
Epoch: 0006 train_loss= 1.8619 train_acc= 0.4214 val_loss= 1.8763 val_acc= 0.3200 time= 0.0818
Epoch: 0007 train_loss= 1.8452 train_acc= 0.4000 val_loss= 1.8632 val_acc= 0.3033 time= 0.0810
Epoch: 0008 train_loss= 1.8288 train_acc= 0.4143 val_loss= 1.8503 val_acc= 0.3500 time= 0.0801
Epoch: 0009 train_loss= 1.8123 train_acc= 0.4429 val_loss= 1.8373 val_acc= 0.4067 time= 0.0797
Epoch: 0010 train_loss= 1.7957 train_acc= 0.4714 val_loss= 1.8245 val_acc= 0.4300 time= 0.0792
Epoch: 0011 train_loss= 1.7795 train_acc= 0.4857 val_loss= 1.8118 val_acc= 0.4500 time= 0.0808
Epoch: 0012 train_loss= 1.7639 train_acc= 0.4929 val_loss= 1.7995 val_acc= 0.4633 time= 0.0818
Epoch: 0013 train_loss= 1.7486 train_acc= 0.4786 val_loss= 1.7876 val_acc= 0.4767 time= 0.0815
Epoch: 0014 train_loss= 1.7336 train_acc= 0.4714 val_loss= 1.7761 val_acc= 0.4633 time= 0.0804
Epoch: 0015 train_loss= 1.7188 train_acc= 0.4571 val_loss= 1.7649 val_acc= 0.4267 time= 0.0788
Epoch: 0016 train_loss= 1.7043 train_acc= 0.4500 val_loss= 1.7539 val_acc= 0.4200 time= 0.0789
Epoch: 0017 train_loss= 1.6901 train_acc= 0.4357 val_loss= 1.7431 val_acc= 0.4100 time= 0.0804
Epoch: 0018 train_loss= 1.6761 train_acc= 0.4357 val_loss= 1.7324 val_acc= 0.3933 time= 0.0792
Epoch: 0019 train_loss= 1.6620 train_acc= 0.4214 val_loss= 1.7219 val_acc= 0.3900 time= 0.0795
Epoch: 0020 train_loss= 1.6481 train_acc= 0.4214 val_loss= 1.7115 val_acc= 0.3867 time= 0.0840
Epoch: 0021 train_loss= 1.6340 train_acc= 0.4286 val_loss= 1.7012 val_acc= 0.3800 time= 0.0788
Epoch: 0022 train_loss= 1.6199 train_acc= 0.4286 val_loss= 1.6911 val_acc= 0.3800 time= 0.0786
Epoch: 0023 train_loss= 1.6059 train_acc= 0.4286 val_loss= 1.6810 val_acc= 0.3833 time= 0.0790
Epoch: 0024 train_loss= 1.5920 train_acc= 0.4286 val_loss= 1.6712 val_acc= 0.3833 time= 0.0790
Epoch: 0025 train_loss= 1.5781 train_acc= 0.4286 val_loss= 1.6614 val_acc= 0.3867 time= 0.0792
Epoch: 0026 train_loss= 1.5643 train_acc= 0.4286 val_loss= 1.6517 val_acc= 0.3933 time= 0.0794
Epoch: 0027 train_loss= 1.5503 train_acc= 0.4357 val_loss= 1.6420 val_acc= 0.3967 time= 0.0810
Epoch: 0028 train_loss= 1.5359 train_acc= 0.4357 val_loss= 1.6321 val_acc= 0.4000 time= 0.0798
Epoch: 0029 train_loss= 1.5214 train_acc= 0.4429 val_loss= 1.6220 val_acc= 0.4067 time= 0.0783
Epoch: 0030 train_loss= 1.5067 train_acc= 0.4714 val_loss= 1.6120 val_acc= 0.4100 time= 0.0787
Epoch: 0031 train_loss= 1.4920 train_acc= 0.4857 val_loss= 1.6020 val_acc= 0.4167 time= 0.0787
Epoch: 0032 train_loss= 1.4773 train_acc= 0.5000 val_loss= 1.5918 val_acc= 0.4200 time= 0.0792
Epoch: 0033 train_loss= 1.4625 train_acc= 0.5143 va